In [81]:
import pickle
import os
import pandas as pd
import numpy as np
import re
import itertools
import time
import pickle
import math
from math import sin, cos, sqrt, atan2, radians
from IPython.display import clear_output

In [82]:
#select marker
marker = 'cytb' # co1

In [83]:
# Load file with filtered sequences
coords = pd.read_csv('data/{}_coordinates.csv'.format(marker), index_col=0)
#coords.dropna(inplace=True)
mammsSP = set(coords.species)

len(mammsSP)

1437

### Function to calculate GD per species and spatial unit

$\widehat{\Pi} = \frac{1}{n \choose 2} \displaystyle\sum_{i=1}^{n-1} \sum_{j=i+1}^{n} \frac{k_{ij}} {m_{ij}}$

$n$ is the number of sequences of the species in the cell

$k_{ij}$ is the number of different nucleotides between sequence i and sequence j

$m_{ij}$ is the number of shared base pairs between sequence i and j

In [84]:
def calculateSpGD(fastaSegs, cellSeqs, includedSeqs, GDthrs = 1):
    pairGD = []
    pairDst = []
    nOfSeqs = set()
    newCellSeqs = [s for s in cellSeqs if s in ''.join(includedSeqs)]
    sequenceComb = list(itertools.combinations(newCellSeqs, 2))
    if len(sequenceComb) > 0:
        for combo in sequenceComb:
            seq1 = re.search('{}.*\n(.*?)\n'.format(combo[0]), fastaSegs).group(1)
            seq2 = re.search('{}.*\n(.*?)\n'.format(combo[1]), fastaSegs).group(1)
            siteList = list(zip(seq1, seq2))
            overlapLen = len([i for i in siteList if '-' not in i and 'N' not in i])
            if overlapLen >= 0.5*max([len(seq1.replace('-',  '').replace('N', '')), len(seq2.replace('-',  '').replace('N', ''))]): #if sequences overlap in at least x% of the longest sequence
                NumberOfSNPs = sum([1 for i in siteList if '-' not in i and 'N' not in i and len(set(i)) > 1])
                OnePairGD = NumberOfSNPs / overlapLen
                if OnePairGD < GDthrs: #maximum GD per pair
                    pairGD.append(OnePairGD)
                    nOfSeqs.add(combo[0])
                    nOfSeqs.add(combo[1])
                    #get distance
                    xc1 = coords.loc[combo[0]]['x']
                    yc1 = coords.loc[combo[0]]['y']
                    xc2 = coords.loc[combo[1]]['x']
                    yc2 = coords.loc[combo[1]]['y']
                    dst = getDistanceHarv(xc1, yc1, xc2, yc2, 6371)
                    pairDst.append(dst)
        if len(pairGD) == 0:
            return np.nan, np.nan, []
        else:
            return pairGD, pairDst, nOfSeqs
    else:
        return np.nan, np.nan, []
    
    
#functions for geographic distance
def deg2rad(deg):
    return deg * (math.pi/180)

def getDistanceHarv(lon1, lat1, lon2, lat2, earthRadius):
    lo1, la1, lo2, la2 = radians(lon1), radians(lat1), radians(lon2), radians(lat2)
    
    R = earthRadius # Radius of the earth in km
    dlon = deg2rad(lon2-lon1)
    dlat = deg2rad(lat2-lat1)
   
    a = sin(dlat / 2)**2 + cos(la1) * cos(la2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    d = R * c
    return d;

### Calculate GD per species and grid-cell

In [85]:
### load python dictionary with cell id sequences per cell. This dictionary was created in the previous step (1_AssignSequencesToSpatialUnits.ipynb)
sd = pickle.load(open( 'data/assigned_sequences/seqsPerGrid_{}.p'.format(marker), "rb" ))    
#new dictionary to store GD for mammals
GDperSPmamms = {}
#get the all filtered sequences
inSeqs = coords.index

start_time = time.time()
totCells = len(sd)
cellGD = {}

for ci, cell in enumerate(sd):
    clear_output(wait=True)
    print('cell {} of {}'.format(ci+1, totCells))

    mammalGD = []
    NmammalSpecies = 0
    NmammalSeqs = 0
    amphGD = []
    NamphSpecies = 0
    NamphSeqs = 0

    for species in sd[cell]:
        # select species with at least 2 sequences
        if len(sd[cell][species]) > 1:
            if species in mammsSP:
                #load alignment
                with open('data/mammals_{}/{}.fasta'.format(marker, species)) as inf:
                    data=inf.read()
                    allPairGD, allPairDist, totSeqs = calculateSpGD(data, sd[cell][species], inSeqs)
                    #store n of seqs and GD
                    GDperSPmamms.setdefault(cell, {})[species] = {'nOfSeqs': totSeqs, 'GD': allPairGD, 'Dist': allPairDist}


print('--- %s seconds ---\n\n\n' % (time.time() - start_time))

#save dictionary
#with open('data/GD_cells_{}.p'.format(marker), 'wb') as fp:
#    pickle.dump(GDperSPmamms, fp, protocol=pickle.HIGHEST_PROTOCOL)

cell 928 of 928
--- 778.8917970657349 seconds ---





In [ ]:
# Data per grid cell can be summarized using the following lines
#get mean GD
meanGD = {c: np.mean([np.nanmean(GDperSPmamms[c][s]['GD']) for s in GDperSPmamms[c] if isinstance(GDperSPmamms[c][s]['GD'], list) ] ) for c in GDperSPmamms }
#get number of sampled species
mammalTaxa = {c: len([GDperSPmamms[c][s] for s in GDperSPmamms[c] if isinstance(GDperSPmamms[c][s]['GD'], list) ]) for c in GDperSPmamms}
#get number of sampled sequences
mammalSeqs = {c: np.sum([len(GDperSPmamms[c][s]['nOfSeqs']) for s in GDperSPmamms[c] if isinstance(GDperSPmamms[c][s]['GD'], list) ]) for c in GDperSPmamms}

### Calculate GD per species and Wallace region

In [89]:
sd = pickle.load(open( 'data/assigned_sequences/seqsPerWallaceRegion_{}.p'.format(marker), 'rb' ))

start_time = time.time()
totRegions = len(sd)
#new dictionary to store GD for mammals
wallGD = {}

#get the all filtered sequences
inSeqs = coords.index

for wi, regio in enumerate(sd):
    print('Region {} of {}: {}'.format(wi+1, totRegions, regio))

    for species in sd[regio]:
        # select species with at least 2 sequences
        if len(sd[regio][species]) > 1:
            if species in mammsSP:
                #load alignment
                with open('data/mammals_{}/{}.fasta'.format(marker, species)) as inf:
                    data=inf.read()
                    allPairGD, allPairDist, totSeqs = calculateSpGD(data, sd[regio][species], inSeqs)
                    #store n of seqs and GD
                    wallGD.setdefault(regio, {})[species] = {'nOfSeqs': totSeqs, 'GD': allPairGD, 'Dist': allPairDist}
                    
    print(np.mean([np.nanmean(wallGD[regio][s]['GD']) for s in wallGD[regio] if isinstance(wallGD[regio][s]['GD'], list) ] ))
                    
#save dictionary
with open('data/GD_wallace_{}.p'.format(marker), 'wb') as fp:
    pickle.dump(wallGD, fp, protocol=pickle.HIGHEST_PROTOCOL)